# [Day 21](https://adventofcode.com/2020/day/21): Allergen Assessment

In [1]:
import re

with open("../data/21.txt", "r") as f:
    foods = re.findall(r"(.*) \(contains (.*)\)", f.read())
    foods = [(ing.split(), agn.split(", ")) for ing, agn in foods]
    ings, agns = zip(*foods)

## Part 1

In [2]:
import numpy as np

def danger(agns, ings):
    hasagn, _   = subsets(agns)
    hasing, ing = subsets(ings)
    dangerous = choice(maybedangerous(hasagn, hasing))
    return hasing, ing, dangerous

def subsets(xss):
    xs = sorted(set.union(*map(set, xss)))
    enum = {x: i for i, x in enumerate(xs)}
    ss = np.zeros((len(xss), len(xs)), dtype=int)
    for i, xs_ in enumerate(xss):
        ss[i, [enum[x] for x in xs_]] = 1
    return ss, np.array(xs)

def maybedangerous(hasagn, hasing):
    cnt = hasagn.T @ hasing
    return cnt == cnt.max(axis=1, keepdims=True)

def choice(x):
    sec, i = np.empty((len(x), 2), dtype=int), 0
    idx = np.einsum("ijk->jki", np.indices(x.shape))
    while len(x):
        j = x.sum(axis=1) == 1
        k = x[j].any(axis=0)
        sec[i:(i := i + j.sum())] = idx[j][x[j]]
        x, idx = x[~j][:, ~k], idx[~j][:, ~k]
    return sec[:, 1][sec[:, 0].argsort()]

hasing, ing, dangerous = danger(agns, ings)
assert 1882 == hasing.sum() - hasing[:, dangerous].sum()

## Part 2

In [3]:
canonicaldangerous = ",".join(ing[dangerous])
assert "xgtj,ztdctgq,bdnrnx,cdvjp,jdggtft,mdbq,rmd,lgllb" == canonicaldangerous